In [26]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import joblib



In [2]:
df = pd.read_csv('./data/raw/mental_health_dataset.csv') 
print("Forma del dataset:", df.shape)

Forma del dataset: (10000, 14)


In [15]:
target_variable = 'mental_health_risk'
if target_variable not in df.columns:
    raise ValueError(f"Target variable '{target_variable}' not found in the dataset.")
X_features_df = df.drop(target_variable, axis=1, errors='ignore')

numerical_features = X_features_df.select_dtypes(include=np.number).columns.tolist()
categorical_features = X_features_df.select_dtypes(include=['object', 'category']).columns.tolist()

Generamos aleatoreamente un dataset con datos nulos utilizando el mecanismo MCAR (Missing Completely At Random).
Esto significa que los valores faltantes se introducen de manera completamente aleatoria, sin depender de ninguna variable observada o no observada del dataset.
Así, cualquier celda tiene la misma probabilidad de ser nula, simulando un escenario donde la ausencia de datos no está relacionada con el resto de la información.

In [8]:
# Crea una copia para trabajar
df_mcar = df.copy()

seed = 42 # Definir la semilla para reproducibilidad

missing_percentage = 0.05  # 5%  Porcentaje de valores nulos a introducir

# Columnas a las que se les aplicará el MCAR
columns_to_corrupt = [col for col in df.columns if col != 'mental_health_risk']

np.random.seed(seed)

for col in columns_to_corrupt:
    mask = np.random.rand(len(df_mcar)) < missing_percentage
    df_mcar.loc[mask, col] = np.nan

display(df_mcar.info())
missing = df_mcar.isnull().mean().sort_values(ascending=False)
missing = missing[missing > 0]
print("Variables con valores nulos:")
display(missing)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   age                     9507 non-null   float64
 1   gender                  9504 non-null   object 
 2   employment_status       9518 non-null   object 
 3   work_environment        9486 non-null   object 
 4   mental_health_history   9507 non-null   object 
 5   seeks_treatment         9482 non-null   object 
 6   stress_level            9519 non-null   float64
 7   sleep_hours             9522 non-null   float64
 8   physical_activity_days  9489 non-null   float64
 9   depression_score        9507 non-null   float64
 10  anxiety_score           9496 non-null   float64
 11  social_support_score    9499 non-null   float64
 12  productivity_score      9467 non-null   float64
 13  mental_health_risk      10000 non-null  object 
dtypes: float64(8), object(6)
memory usage: 

None

Variables con valores nulos:


productivity_score        0.0533
seeks_treatment           0.0518
work_environment          0.0514
physical_activity_days    0.0511
anxiety_score             0.0504
social_support_score      0.0501
gender                    0.0496
depression_score          0.0493
age                       0.0493
mental_health_history     0.0493
employment_status         0.0482
stress_level              0.0481
sleep_hours               0.0478
dtype: float64

Existen dos enfoques principales para tratar los datos faltantes:

1. **Eliminación de filas:** Consiste en eliminar todas las filas que contienen al menos un valor nulo. Aunque este método es sencillo y garantiza que el dataset resultante no tenga valores faltantes, puede ser demasiado drástico, ya que se pierde información valiosa y puede reducir significativamente el tamaño de la muestra.

2. **Imputación:** Es el proceso de reemplazar los valores faltantes (NaN) por valores estimados o sustitutos. La imputación permite "rellenar" los huecos en los datos, facilitando su uso en análisis estadísticos o modelos de machine learning, los cuales generalmente no pueden manejar valores nulos directamente. Existen diversas técnicas de imputación, desde métodos simples como la media o la moda, hasta técnicas más avanzadas como la imputación múltiple o el uso de modelos predictivos.

### 1. Eliminación de Filas

In [13]:
# Elimina las filas con cualquier valor nulo
df_sin_nulos = df_mcar.dropna()

# Muestra la forma y las primeras filas del nuevo dataset
print("Forma del dataset sin nulos:", df_sin_nulos.shape)
if (df_sin_nulos.shape[0] > 5000):
    print("✔️ El dataset es grande y cumple con los requisitos de la práctica.")
else:
    print("❌ El dataset es pequeño y no cumple con los requisitos de la práctica.")
# display(df_sin_nulos.head())

Forma del dataset sin nulos: (5130, 14)
✔️ El dataset es grande y cumple con los requisitos de la práctica.


### 2.Imputación

In [18]:
df_inputed = df_mcar.copy()

for col in numerical_features:
    median_val = df_inputed[col].median()
    df_inputed[col] = df_inputed[col].fillna(median_val)

for col in categorical_features:
    mode_val = df_inputed[col].mode()[0]
    df_inputed[col] = df_inputed[col].fillna(mode_val)

df_inputed.dropna(subset=[target_variable], inplace=True) # This inplace is fine as it's on the DataFrame itself
print("Forma: ", df_inputed.shape)


Forma:  (10000, 14)


---
Cuando tienes datos faltantes (NaN), hay dos caminos principales: **imputar** (rellenarlos) o **borrarlos**.

La **imputación** es, en general, la **mejor opción**. Consiste en estimar y rellenar los datos que faltan (usando la mediana para números o la moda para categorías, como haces en tu código). Esto es clave porque **conserva casi toda tu información**, evitando que tu dataset se reduzca y se introduzca sesgos que podrían aparecer al eliminar filas, especialmente si los datos no faltan de forma totalmente aleatoria. Además, la mayoría de los modelos de Machine Learning necesitan datos completos.

**Borrar datos** (usando `dropna()`) es más drástico. Se usa solo si tienes **muy pocos datos faltantes** (menos del 1-2%) y estás seguro de que faltan completamente al azar, o si una columna tiene **demasiados** datos faltantes (más del 70-80%) y ya no es útil. La única excepción crucial es cuando la **variable que quieres predecir (tu objetivo)** tiene valores faltantes; en ese caso, sí se deben borrar esas filas, ya que no puedes entrenar un modelo sin saber qué debe predecir.

Tu método actual (imputar características y solo borrar filas si la variable objetivo está ausente) es un **excelente equilibrio** que maximiza la información y prepara tus datos de forma robusta para el modelado.
Aquí tienes un resumen de la explicación sobre la imputación vs. borrar datos faltantes:

---

## Borrar vs. Imputar Datos Faltantes

Cuando tienes datos faltantes (NaN), hay dos caminos principales: **borrarlos** o **imputarlos** (rellenarlos).

### Borrar Datos (`dropna()`)

* **¿Qué es?** Eliminar filas o columnas enteras que contengan valores faltantes.
* **Ventajas:** Es simple y rápido de implementar.
* **Desventajas:** Causa **pérdida de información** y **reduce el tamaño de tu conjunto de datos**, lo que puede llevar a resultados sesgados si los datos no faltan completamente al azar. Generalmente, solo es una buena opción si tienes **muy pocos NaN** o si una columna tiene **demasiados NaN** para ser útil.

### Imputar Datos

* **¿Qué es?** Reemplazar los valores faltantes con estimaciones (como la mediana para números, o la moda para categorías).
* **Ventajas:** **Mantiene más datos**, conservando el tamaño de la muestra y la potencia estadística. Puede **reducir el sesgo** si los datos faltantes no son completamente aleatorios. Hace que tus datos sean compatibles con la mayoría de los modelos de Machine Learning.
* **Desventajas:** Los valores imputados son solo estimaciones y pueden introducir un pequeño "ruido" o incertidumbre.

---

### Recomendación General:

En la mayoría de los casos de Machine Learning, **la imputación es la opción preferida**. Te permite aprovechar al máximo tus datos y evitar la pérdida de información crucial. La estrategia más común y balanceada es **imputar las características (variables de entrada)** y **solo borrar filas si el valor faltante está en la variable objetivo (lo que quieres predecir)**.

In [23]:
df_processed = df_inputed.copy()

In [24]:
y_series = df_processed[target_variable]
le_target = LabelEncoder()
y = le_target.fit_transform(y_series)

In [29]:
X = df_processed[numerical_features + categorical_features]

In [30]:
high_cardinality_threshold = 15 #define un threshold para alta cardinalidad (15 valores únicos)
high_cardinality_cols = [col for col in categorical_features if X[col].nunique() > high_cardinality_threshold]
#esto se hace para evitar problemas de memoria y rendimiento con OneHotEncoder


In [31]:
final_numerical_features = [col for col in numerical_features if col in X.columns]
final_categorical_features = [col for col in categorical_features if col in X.columns]

transformers_list = []


transformers_list.append(('num', StandardScaler(), final_numerical_features))


transformers_list.append(('cat', OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False), final_categorical_features))

preprocessor = ColumnTransformer(
    transformers=transformers_list,
    remainder='passthrough'
)

In [ ]:

# Guardar el preprocesador
joblib.dump(preprocessor, './models/preprocessor.joblib')

# Guardar los LabelEncoders (si los usaste, por ejemplo, para la variable objetivo)
# Si usaste LabelEncoder en alguna columna, deberías guardarlos así:
joblib.dump(le_target, './models/label_encoder_target.joblib')

# Para cargar en otro notebook:
# preprocessor = joblib.load('preprocessor.joblib')
# label_encoder_target = joblib.load('./data/processed/label_encoder_target.joblib')

['./data/processed/label_encoder_target.joblib']

In [ ]:
# Guardar el dataset procesado
df_processed.to_csv('./data/processed/mental_health_dataset_processed.csv', index=False)
print("Dataset procesado guardado en './data/processed/mental_health_dataset_processed.csv'.")

Dataset procesado guardado en './data/processed/mental_health_dataset_processed.csv'.


---

In [ ]:
# Guardar X e Y procesados (después de imputación, antes de escalado/codificación)
# Esto es útil para tener un punto de partida "limpio" sin NaN para otros notebooks.
X.to_csv('./data/processed/X_processed.csv', index=False)
pd.DataFrame(y, columns=[target_variable]).to_csv('./data/processed/y_processed.csv', index=False)
print("X e y procesados (imputados) guardados en './data/processed/'.")

X e y procesados (imputados) guardados en './data/processed/'.
